In [ ]:

import numpy as np
from skimage.color import rgb2gray,rgba2rgb
from skimage.measure import find_contours
from skimage import transform, feature, color , morphology , measure, segmentation, filters, draw, io
from commonfunctions import * 
from skimage.draw import rectangle

%matplotlib inline
%load_ext autoreload
%autoreload 2


## Read Imag and handle its orientation

In [ ]:
# Read the imag

def deskew_image(img):
    # Read the image
    image = np.copy(img)
    gray = color.rgb2gray(image)

    # Apply edge detection using Canny
    edges = feature.canny(gray, sigma=1.0, low_threshold=0.1, high_threshold=0.9)

    # Use probabilistic Hough Transform to detect lines
    lines = transform.probabilistic_hough_line(edges, threshold=10, line_length=5, line_gap=3)
    
    # Determine the dominant angle of the lines
    angles = [np.arctan2((y2 - y1), (x2 - x1)) for (x1, y1), (x2, y2) in lines]
    dominant_angle = np.median(angles) * 180 / np.pi

    # Rotate the image to correct skewing and orientation
    rotated_image = transform.rotate(image, dominant_angle, resize=True)

    return rotated_image
# image 4,15 is the only one that is rotated wrong 

deskewed_images = list()

# for i in range(10,15):
#     image = io.imread('grade_sheet/{}.jpg'.format(i))
#     deskewed_image = deskew_image(image)
#     show_images([image,deskewed_image], ['image', 'deskewed_image'])
#     deskewed_images.append(deskewed_image)
    

## Crop the image

In [ ]:
def draw_contours(img):
    # Apply Gaussian blur
    image = np.copy(img)
    
    blurred = color.rgb2gray(image)
    blurred = blurred > .5

    # Perform morphological operations to close gaps in edges
    blurred = morphology.closing(blurred, morphology.square(6))

    # Label connected components (no effect)
    label_image = measure.label(blurred)

    # Find contours using scikit-image
    cnts = measure.find_contours(label_image, .5)

    max_area = 0
    max_box = None
    for cnt in cnts:
        Xmin = int(round(min(cnt[:,1])))
        Xmax = int(round(max(cnt[:,1])))
        Ymin = int(round(min(cnt[:,0])))
        Ymax = int(round(max(cnt[:,0])))

        width = abs( Xmax - Xmin) 
        height = abs(Ymax - Ymin)
        area = width * height 
        
        if area > max_area:
            max_area = area 
            max_box = [Xmin, Xmax, Ymin, Ymax]

    [Xmin, Xmax, Ymin, Ymax] = max_box
    print(max_box)
    rr, cc = rectangle(start = (Ymin,Xmin), end = (Ymax,Xmax), shape=image.shape)
    image[rr, cc] = [255,255,255] #set color white
    return max_box, image



# for i in range(1,10):
#     image = io.imread('grade_sheet/{}.jpg'.format(i))
#     blurred, mask = draw_contours(image)
#     sheet = mask
#     show_images([image,mask, sheet ], ['image', 'mask', 'sheet'])


## Perspective Transform

In [ ]:

def perspective_transform_image(img, margin = 30):
    # Read the image
    image = np.copy(img)
    
    # get the source points 
    image = deskew_image(image)
    box, mask = draw_contours(image)
    [xmin, xmax, ymin, ymax]= box 

    # Convert the image to grayscale
    gray_image = color.rgb2gray(image)

    source_points = np.array( [[0, 0] ,[3000, 0 ] ,[0, 3000] ,[3000 ,3000]] )

    # Define the destination points (a rectangle)
    destination_points = np.array([[xmin+margin, ymin+margin],  [xmax-margin, ymin+margin], [xmin+margin,ymax-margin], [xmax-margin, ymax-margin]])

    # Estimate the perspective transform
    tform = transform.ProjectiveTransform()
    tform.estimate(source_points, destination_points)

    # Apply the perspective transformation to the image
    transformed_image = transform.warp(image, tform, output_shape=(3000, 3000))

    return transformed_image


# for i in range(1,2):
#     image = io.imread('grade_sheet/{}.jpg'.format(i))
#     show_images([image,perspective_transform_image(image)], ['image', 'perspective_transform_image'])

## OCR

In [ ]:
image = io.imread('grade_sheet/1.jpg')
processed_image = perspective_transform_image(image,   )
#show_images([image,processed_image], ['image', 'processed_image'])

image = np.copy(processed_image)
blurred = rgb2gray(image)
blurred = blurred > .5
blurred = morphology.closing(blurred, morphology.square(6))
label_image = measure.label(blurred)
# Find contours using scikit-image
cnts = measure.find_contours(label_image, .5)

max_area = 0
max_box = None
for cnt in cnts:
    Xmin = int(round(min(cnt[:,1])))
    Xmax = int(round(max(cnt[:,1])))
    Ymin = int(round(min(cnt[:,0])))
    Ymax = int(round(max(cnt[:,0])))

    width = abs( Xmax - Xmin) 
    height = abs(Ymax - Ymin)
    area = width * height 
    
    
    if area > max_area:
        max_area = area 
        max_box = [Xmin, Xmax, Ymin, Ymax]

    rr, cc = rectangle(start = (Ymin,Xmin), end = (Ymax,Xmax), shape=image.shape)
    image[rr, cc] = 1 


#show_images([image], ['img'])


In [9]:

import pytesseract 
# change this if you are not running on windows 
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"


image = io.imread('grade_sheet/1.jpg')

pytesseract.image_to_string(image, lang='ara', config='.')



'(7065037)11:13 : 11006 000 - الجيزة الر ثيسي - 45 - 20105 : وممنتقم ا (651010326) عصتاعبا 112 مساععا عم كنا وق 2021-ااوع\n\n112008 الكناعمع]\n\nتلات ...1 سمي أستسيع 2076 500201 | |\n17 ل\nالح ونال ار لوو وي\nامات فاو و ل ا\n11880655\n\nيه\n0\n\nاع 6331 1020100606 ١ادااحروطح\u200f\n\nعبد الله محمد جلال السحيمى | 1180255\nا ا أ \u200e١\u200f كمسا ممه سم عم الاي عبس ع\nا 1 الست وير\n\u200e١\u200f ك2 | بدي بستقادة رهم اوماد دوه أعير مسد قي هلقي خبروى .| ينوع\nالأ ان لالت ل 5 ربج و\nفجت معلا 2610 0553003 122\nا ا \u200e١\u200f شه حا وه دم ل اله ميب |لبصسم\nا 0 رس مان ل تور لتب ور\n\u200e١ 11\u200f خنطا ا رمه ته مسيسي اننا اانا سند قشي | دمت\n1 شطع 2 222 1105:2123 116002 11/105132 أمصطفى نخذ فى عزت حافظ\nلاا رثن الوك مر ا أل تقار المسميسي و\n7 20665 لاط21] 2واح ك2166/يا\n\nالحا وداىيا\nايا\n\nس0\n1\n\n0\n1\n\nاح\n\nتا\n\nلد\n\nلاك\n541\n\n12 ع3\n\n'